In [1]:
from pathlib import Path
from topagnps2cche1d import Watershed

In [2]:
# root_dir = Path("../data_inputs/goodwin_creek_1m/")
root_dir = Path(
    "../data_inputs/goodwin_creek_1m_1_and_3_reaches/TopAGNPS_DataSets/3_Reach"
)
# root_dir = Path("../data_inputs/topagnps_ohio_files/")

path_to_reaches = root_dir / "AnnAGNPS_Reach_Data_Section.csv"
path_to_cells = root_dir / "AnnAGNPS_Cell_Data_Section.csv"
path_to_reaches_raster = root_dir / "AnnAGNPS_Reach_IDs.asc"

In [3]:
watershed = Watershed()
watershed.import_topagnps_reaches_network(path_to_reaches)
watershed.import_topagnps_cells(path_to_cells)

watershed.read_reaches_geometry_from_topagnps_asc_file(path_to_reaches_raster)

watershed.assign_strahler_number_to_reaches()
watershed.ignore_reaches_with_strahler_leq(0)

watershed.identify_inflow_sources()
watershed.update_junctions_and_node_types()

watershed.renumber_all_nodes_and_reaches_in_CCHE1D_computational_order()
watershed.set_node_id_to_compute_id()

Reading Reaches: 100%|██████████| 3/3 [00:00<00:00, 79.56it/s]


In [4]:
reaches = watershed.reaches
for k, v in reaches.items():
    # if not v.ignore:
    print(v)
# break

-----------------------------
Reach            : 2
CCHE1D Ch. ID    : 3
Receiving Reach  : 1
Slope            : 0.01716
Strahler Number  : 2
Ignore?          : False
(US/DS) Nodes    : (356, 385)
-----------------------------
Reach            : 3
CCHE1D Ch. ID    : 2
Receiving Reach  : 2
Slope            : 0.01082
Strahler Number  : 1
Ignore?          : False
(US/DS) Nodes    : (280, 355)
-----------------------------
Reach            : 4
CCHE1D Ch. ID    : 1
Receiving Reach  : 2
Slope            : 0.01332
Strahler Number  : 1
Ignore?          : False
(US/DS) Nodes    : (1, 279)


In [5]:
reach2 = watershed.reaches[2]
reach3 = watershed.reaches[3]
reach4 = watershed.reaches[4]

## Plotting network

In [37]:
watershed.plot(frame_height=700, frame_width=1200, line_width=2, by='Reach_ID', title='Watershed')

:Overlay
   .NdOverlay.I  :NdOverlay   [Reach_ID]
      :Curve   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID)
   .NdOverlay.II :NdOverlay   [Reach_ID]
      :Scatter   [X]   (Y,TYPE,US2ID,USID,ID,COMPUTEID,DSID)

In [137]:
import holoviews as hv
import numpy as np
from scipy.interpolate import make_interp_spline, interp1d

In [168]:
x, y = reach2.get_x_y_node_arrays_us_ds_order()
scatter_line_og = (hv.Scatter((x,y)).opts(aspect='equal', size=8, frame_width=1200) * hv.Curve((x,y)))

# interp
p = np.stack((x,y))
dp = p[:,1:] - p[:, :-1] # 2 vector distance between points
l = (dp**2).sum(axis=0)        # squares of lengths of 2-vectors between points
u_cord = np.sqrt(l).cumsum()   # cumulative sums of 2-norms
u_cord = np.r_[0, u_cord]      # the first point is parameterized at zero
u_c = np.r_[0, np.cumsum((dp**2).sum(axis=0)**0.25)] # centripetal

spl_cord = make_interp_spline(u_cord, np.c_[x,y])
# spl_cord = interp1d(u_cord, p)
# spl_c = make_interp_spline(u_c, p, axis=1)    # note p is a 2D array
spl_c = make_interp_spline(u_c, np.c_[x,y])    # note p is a 2D array

# uu = np.linspace(u_cord, p, 51)

numpoints = 5
uu = np.linspace(u_cord[0], u_cord[-1], numpoints)
xx_cord, yy_cord = spl_cord(uu).T

uu = np.linspace(u_c[0], u_c[-1], numpoints)
xx_c, yy_c = spl_c(uu).T

scatter_line_cord = (hv.Scatter((xx_cord, yy_cord)).opts(aspect='equal', size=8, frame_width=1200, frame_height=800) * hv.Curve((xx_cord,yy_cord)))
scatter_line_c = (hv.Scatter((xx_c, yy_c)).opts(aspect='equal', size=8, frame_width=1200) * hv.Curve((xx_c,yy_c)))

# scatter_line_og 

# scatter_line_c * scatter_line_cord

scatter_line_og * scatter_line_cord * scatter_line_c

:Overlay
   .Scatter.I   :Scatter   [x]   (y)
   .Curve.I     :Curve   [x]   (y)
   .Scatter.II  :Scatter   [x]   (y)
   .Curve.II    :Curve   [x]   (y)
   .Scatter.III :Scatter   [x]   (y)
   .Curve.III   :Curve   [x]   (y)

In [165]:
len(x)

30

In [100]:
L = [1,2,3,4,5,6,7]
print(L[::-1])

[7, 6, 5, 4, 3, 2, 1]


In [46]:
df2 = reach2.get_nodes_as_df()

df2.hvplot.line(x='X', y='Y', by='Strahler_Number', aspect='equal', line_width=2)


:NdOverlay   [Strahler_Number]
   :Scatter   [X]   (Y)

:NdOverlay   [Strahler_Number]
   :Scatter   [X]   (Y)